# Overview

OpenELM is a open-source model. It is open-source from head to toes which not only the weights, but also the traeining data, hyperparameters and code. This 270M parameter variant is the smallest modern GPT with long context(2048 tokens) that is actually usable.

Accoridng to Apple, it uses a layer-wise scaling strategy to efficiently allocate parameters within each layer of the transformer model. This concept comes from the DeLighT: Deep and Light-weight Transformer paper.

In [1]:
!pip install -U -q transformers==4.39.3
!pip install -U -q accelerate==0.28.0

In [2]:
import os
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["TOKENIZER"]="meta-llama/Llama-2-7b-hf"
os.environ["MODEL"]="apple/OpenELM-270M-Instruct"

# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(os.getenv("TOKENIZER"))

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(os.getenv("MODEL"), trust_remote_code=True)
model.eval()
model.to("cuda")
model.device

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

configuration_openelm.py:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M-Instruct:
- configuration_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_openelm.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/apple/OpenELM-270M-Instruct:
- modeling_openelm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

device(type='cuda', index=0)

In [5]:
from transformers import TextStreamer

# https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.TextStreamer
streamer=TextStreamer(tokenizer)

In [6]:
prompt="The weather in Melbourne is pretty good today"

tokenized_prompt=tokenizer(prompt, return_tensors='pt').to("cuda")

In [7]:
_=model.generate(**tokenized_prompt, 
               streamer=streamer, 
               max_new_tokens=200, 
               pad_token_id=0,
               repetition_penalty= 1.2,
               do_sample=True,
               temperature=0.1, top_k=50)

2024-05-18 13:18:26.135283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 13:18:26.135382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 13:18:26.259428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<s> The weather in Melbourne is pretty good today, with a high temperature of 31 degrees Celsius and a low temperature of 21 degrees Celsius.
The sun shines brightly through the sky, casting a soft golden glow over the city.
Melbourne's annual Fashion Festival kicks off this weekend, showcasing some of the hottest designers from around the globe.
This year's edition will feature runway shows by emerging designers such as Tanya Burr, Yves Klein, Alexander McQueen, Diane von Furstenberg, and Karl Lagerfeld.
Fashion Week Melbourne has also announced its first-ever Designer Spotlight, featuring an exclusive interview with each designer.
Tanya Burr, who won the Miss Universe Australia beauty pageant last year, will be speaking at the event.
"I love being able to share my story with other women," Burr told Vogue Australia.



# Acknowledge

* https://medium.com/the-ai-explorer/omg-270-millions-small-apple-to-do-what-1db9eb0601f1

# Reference

* https://github.com/apple/corenet/tree/main/mlx_examples/open_elm